# Let's make a a quiz? 
The homework is to create a quiz app. We will be basing it on the tutorial described in:  https://radiusofcircle.blogspot.com/2016/03/making-quiz-website-with-python.html

The first step in our quiz app is make a quiz program that works without the web. Once we have a working quiz program, then we will package it as an app.

# Let's start very very small - matching, 3 questions so we can test:
1. What is the programming language used in this class? Python, Assembly, Java, Lisp
2. How many homework assignments have been posted? 1, 3, 4, 2
3. What day of the week is it? Monday, Wednesday, Friday, Sunday

# What's a convenient way to encode the questions? Spreadsheet

In [20]:
import pandas as pd
df = pd.read_csv('quizdata.csv')
df

,question,opt1,opt2,opt3,opt4,answer
0,What is the programming language used in this ...,Python,Assembly,Java,Lisp,Python
1,How many homework assignments have been posted?,1,3,4,2,2
2,What day of the week is it?,Monday,Wednesday,Friday,Sunday,Friday


# Quiz & Scorer Functions
Let's write a quiz function that randomly selects a question, asks the user to choose an answer, and checks it against the correct answer. We start by using 1 question. 

Coding Notes:
1. df.iloc[i] - get the integer location (row) of the dataframe

In [43]:
df.iloc[0]

question    What is the programming language used in this ...
opt1                                                   Python
opt2                                                 Assembly
opt3                                                     Java
opt4                                                     Lisp
answer                                                 Python
Name: 0, dtype: object

2. 'opt1':'opt4' is Pandas syntax (code) to get all the columns that are next to each other. This syntax only works when we have a single row,

In [44]:
df.iloc[0]['opt1':'opt4']

opt1       Python
opt2     Assembly
opt3         Java
opt4         Lisp
Name: 0, dtype: object

3. " ".join combines the strings with a space. For example:

In [45]:
" ".join(df.iloc[0]['opt1':'opt4'])

' Python  Assembly  Java   Lisp'

4. Use .lower to make our question case insensitive


In [46]:
'Python'.lower()

'python'

In [47]:
import random
def quiz(questions):
    print(questions['question'])
    options = " ".join(questions['opt1':'opt4'])
    ans = input(f"choose 1:{options}: ")
    if ans.lower() == questions['answer'].lower():
        return True
    return False

In [56]:
correct = quiz(df.iloc[0]) # this prints one question
print(f'Right?: {correct}') #uses print formatting to substitute in the value of correct

What is the programming language used in this class?
choose 1: Python  Assembly  Java   Lisp: lisp
Right?: False


Since this is a quiz, we also want to shuffle the questions and answers on each round. Here we do so by randomly sampling all the rows from the data frame

In [33]:
df.sample(frac=1)

,question,opt1,opt2,opt3,opt4,answer
2,What day of the week is it?,Monday,Wednesday,Friday,Sunday,Friday
1,How many homework assignments have been posted?,1,3,4,2,2
0,What is the programming language used in this ...,Python,Assembly,Java,Lisp,Python


We also want to shuffle the order of the questions. First let's loop over the questions user the `iterrows` method, which works like enumerate and gives the row and its location (idx)

In [80]:
for idx, row in df.sample(frac=1).iterrows():
    print(row)
    break # stops the loop after one iteration

question    How many homework assignments have been posted?
opt1                                                      1
opt2                                                      3
opt3                                                      4
opt4                                                     2 
answer                                                    2
Name: 1, dtype: object


Row is similar to a dictionary, so we can access the elements using `row['column name']`

In [81]:
for idx, row in df.sample(frac=1).iterrows():
    print(row['question'])
    print([row['opt1'], row['opt2'], row['opt3'], row['opt4']])
    break

How many homework assignments have been posted?
['1', ' 3', ' 4', ' 2 ']


Python's random library has a sample method that we can use on lists, like the options list here:

In [98]:
import random
for idx, row in df.sample(frac=1).iterrows():
    print(row['question'])
    options = random.sample([row['opt1'], row['opt2'], row['opt3'], row['opt4']],4)
    print(options)
    break

How many homework assignments have been posted?
[' 4', ' 3', '1', ' 2 ']


Because want this quiz to work for multiple questions, let's write a new quiz function that incoporates shuffling and multiple questions into the existing quiz function. We'll also keep count of how many are correct instead of using true and false.

In [30]:
def quiz(questions):
    correct = 0
    
    # keep a count of the answers
    return correct

# How do we make this an app?
It's a form, which is a post request because we're updating data

# We start with the template: how should this be displayed?


```html
<h1>Psych 31170 Quiz</h1>
<form action='/grade' method='POST'>
    <ol>
        {% for q in questions %}
            <li>{{q['question']}}</li>
            {% for o in q['options'] %}
                <input type='radio' value='{{o}}' name='{{loop.index }}'/>{{o}}
            {% endfor %}
        {% endfor %}
    </ol>
    <input type="submit" value="submit" />
</form>
```

Save the above in a questions.html file

Let's repackage the dataframe into a list of dicts that can be posted to the template

In [108]:
import random
questions = []
for idx, row in df.sample(frac=1).iterrows():
    q = dict()
    q['question'] = row['question']
    q['options'] = random.sample([row['opt1'], row['opt2'], row['opt3'], row['opt4']],4)
    questions.append(q)

In [111]:
# what's that look like?
for q in questions:
    print(q['question'])
    for o in q['options']:
        print(o)

What is the programming language used in this class?
 Python
  Lisp
 Assembly
 Java
How many homework assignments have been posted?
 4
1
 3
 2 
What day of the week is it?
 Sunday
 Friday
 Monday
 Wednesday


The name attribute in the radio buttons keeps the group together, and the template code gets stored in a file called quiz.html in the templates folder. Here we name the group based on its location in the questions list using loop.index so that we can refer to the question again when evaluating if the answer is correct.

# How do we talk to our quiz?

```python

@app.route('/quiz')
def quiz():
    //insert your quiz code here
    return render_template('questions.html', questions=questions)
```

In [113]:
from IPython.display import HTML
HTML("http://127.0.0.1:5000/quiz")

<IPython.core.display.HTML object>

# How do we check results? What happens on the submit?
```
<form action='/grade' method='POST'>
```

* request object - request->to form, response comes back
* form is an attribute on quest that returns form[name] where name is identified in the form elements:
```html
     <input type='radio' value='{{o}}' name='{{loop.index }}'/>
```
```python
@app.route('/grade', methods=['POST'])
def quiz_answers():
 correct = 0
 for q in questions:
    if q['answer'] == request.form[q['qid']]:
        corr

 return '<h1>Correct Answers: <u>'+str(correct)+'</u></h1>'
```

In [ ]:
HTML("http://127.0.0.1:5000/quiz")

# Submission Requirements
Take all the files required for this project and submit them as a .zip file
1. Create your own list of questons-must have at least 10 and submit the spreadsheet
2. Work through this notebook. You must submit this notebook with the quiz function implemented in it.
3. Also submit a myapp folder containing the code for your app: 
  1. myapp.py -> Python file with working app code 
  2. template folder, containing 'questions.html'
I should be able to run myapp.py and interact with it the way we have been interacting with the apps we've been writing in class in lecture 8 and 9. 
